In [1]:
!pip install opendatasets
import opendatasets as od
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')
np.seterr(divide = 'ignore')

od.download("https://www.kaggle.com/competitions/passenger-satisfaction/data?select=pass_satisfaction_train.csv")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: bal00p
Your Kaggle Key: ··········


100%|██████████| 1.99M/1.99M [00:00<00:00, 143MB/s]


Extracting archive ./passenger-satisfaction/passenger-satisfaction.zip to ./passenger-satisfaction


In [47]:
df = pd.read_csv(r"/content/passenger-satisfaction/pass_satisfaction_train.csv", sep=',')
df

,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,110028,0,Loyal Customer,26,Business travel,Business,1142,2,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
1,24026,0,Loyal Customer,25,Business travel,Business,562,2,5,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
2,111157,0,Loyal Customer,26,Personal Travel,Eco,1180,3,4,2,...,1,3,4,4,4,4,1,0,0.0,neutral or dissatisfied
3,82113,1,Loyal Customer,47,Personal Travel,Eco,1276,2,4,2,...,2,3,3,4,3,5,2,9,23.0,neutral or dissatisfied
4,79485,0,Loyal Customer,41,Business travel,Business,853,1,2,2,...,1,1,2,1,4,1,2,0,0.0,neutral or dissatisfied
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83118,94171,0,disloyal Customer,23,Business travel,Eco,192,2,1,2,...,2,3,1,4,2,3,2,3,0.0,neutral or dissatisfied
83119,73097,1,Loyal Customer,49,Business travel,Business,2347,4,4,4,...,5,5,5,5,5,5,4,0,0.0,satisfied
83120,68825,1,disloyal Customer,30,Business travel,Business,1995,1,1,1,...,4,3,2,4,5,5,4,7,14.0,neutral or dissatisfied
83121,54173,0,disloyal Customer,22,Business travel,Eco,1000,1,1,1,...,1,4,5,1,5,4,1,0,0.0,neutral or dissatisfied


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83123 entries, 0 to 83122
Data columns (total 24 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   id                                 83123 non-null  int64  
 1   Gender                             83123 non-null  int64  
 2   Customer Type                      83123 non-null  object 
 3   Age                                83123 non-null  int64  
 4   Type of Travel                     83123 non-null  object 
 5   Class                              83123 non-null  object 
 6   Flight Distance                    83123 non-null  int64  
 7   Inflight wifi service              83123 non-null  int64  
 8   Departure/Arrival time convenient  83123 non-null  int64  
 9   Ease of Online booking             83123 non-null  int64  
 10  Gate location                      83123 non-null  int64  
 11  Food and drink                     83123 non-null  int

In [49]:
df_nan = df.loc[:, df.isnull().any()]
df_nan[df_nan.isnull().any(axis=1)]

,Arrival Delay in Minutes
167,NaN
1240,NaN
1618,NaN
2004,NaN
2124,NaN
...,...
81227,NaN
81652,NaN
82040,NaN
82373,NaN


In [50]:
df_nan.isnull().sum() / len(df_nan) * 100

Arrival Delay in Minutes    0.287526
dtype: float64

In [51]:
df['Arrival Delay in Minutes'].value_counts()

0.0      46499
1.0       1739
2.0       1681
3.0       1557
4.0       1540
         ...  
624.0        1
291.0        1
729.0        1
275.0        1
369.0        1
Name: Arrival Delay in Minutes, Length: 430, dtype: int64

Будем считать пропуски, как нулевую задержку.

In [52]:
df['Arrival Delay in Minutes'] = df['Arrival Delay in Minutes'].fillna(0.0)
df['Arrival Delay in Minutes'].value_counts()

0.0      46738
1.0       1739
2.0       1681
3.0       1557
4.0       1540
         ...  
624.0        1
291.0        1
729.0        1
275.0        1
369.0        1
Name: Arrival Delay in Minutes, Length: 430, dtype: int64

Разберемся со строками

In [53]:
print([df[i].unique() for i in df.select_dtypes(include='object').columns])

[array(['Loyal Customer', 'disloyal Customer'], dtype=object), array(['Business travel', 'Personal Travel'], dtype=object), array(['Business', 'Eco', 'Eco Plus'], dtype=object), array(['satisfied', 'neutral or dissatisfied'], dtype=object)]


In [54]:
from sklearn.preprocessing import StandardScaler
from sklearn import datasets
from matplotlib.colors import ListedColormap

X = df.drop(columns=['satisfaction'])
y_train = df[['satisfaction', 'id']].drop(columns=['id'])

In [64]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import make_column_transformer

label_encoder = LabelEncoder()
label_encoder.fit(y_train['satisfaction'])
y_train['satisfaction'] = label_encoder.transform(y_train['satisfaction'])

transformer = make_column_transformer((OneHotEncoder(), X.select_dtypes(include='object').columns), remainder='passthrough')
transformed = transformer.fit_transform(X)
X = pd.DataFrame(transformed)
X

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,110028.0,0.0,26.0,...,5.0,5.0,4.0,3.0,4.0,4.0,4.0,5.0,0.0,0.0
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,24026.0,0.0,25.0,...,2.0,2.0,2.0,5.0,3.0,1.0,4.0,2.0,11.0,9.0
2,1.0,0.0,0.0,1.0,0.0,1.0,0.0,111157.0,0.0,26.0,...,1.0,1.0,3.0,4.0,4.0,4.0,4.0,1.0,0.0,0.0
3,1.0,0.0,0.0,1.0,0.0,1.0,0.0,82113.0,1.0,47.0,...,2.0,2.0,3.0,3.0,4.0,3.0,5.0,2.0,9.0,23.0
4,1.0,0.0,1.0,0.0,1.0,0.0,0.0,79485.0,0.0,41.0,...,3.0,1.0,1.0,2.0,1.0,4.0,1.0,2.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83118,0.0,1.0,1.0,0.0,0.0,1.0,0.0,94171.0,0.0,23.0,...,2.0,2.0,3.0,1.0,4.0,2.0,3.0,2.0,3.0,0.0
83119,1.0,0.0,1.0,0.0,1.0,0.0,0.0,73097.0,1.0,49.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,0.0,0.0
83120,0.0,1.0,1.0,0.0,1.0,0.0,0.0,68825.0,1.0,30.0,...,5.0,4.0,3.0,2.0,4.0,5.0,5.0,4.0,7.0,14.0
83121,0.0,1.0,1.0,0.0,0.0,1.0,0.0,54173.0,0.0,22.0,...,1.0,1.0,4.0,5.0,1.0,5.0,4.0,1.0,0.0,0.0


In [65]:
std = StandardScaler()
X_train = std.fit_transform(X)
X_train = pd.DataFrame(X_train, columns=X.columns)

n = X_train.shape[1]
classes = datasets.make_classification(n_samples=100,
                                       n_features=n,
                                       n_informative=n, 
                                       n_redundant=0, 
                                       n_classes=n, 
                                       random_state=2116)


X_train_tr = X_train.transpose()
y_train_tr = y_train.transpose()

In [59]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [60]:
def log_loss(w, X, y):
    m = X.shape[1]

    A = sigmoid(np.dot(w.T, X))
    
    loss = -1.0 / m * np.sum(y * np.log(A) + (1 - y) * np.log(1 - A))
    loss = np.squeeze(loss)
    grad = 1.0 / m * np.dot(X, (A - y).T)
    
    return loss, grad

In [61]:
def optimize(w, X, y, n_iterations, eta):
    losses = []
    
    for i in range(n_iterations):        
        loss, grad = log_loss(w, X, y)
        w = w - eta * grad

        losses.append(loss)
        
    return w, losses

In [62]:
def predict(w, X, b=0.5):
    
    m = X.shape[1]
    
    y_predicted = np.zeros((1, m))
    w = w.reshape(X.shape[0], 1)
    
    A = sigmoid(np.dot(w.T, X))
    
    for i in range(A.shape[1]):
        if (A[:,i] > b): 
            y_predicted[:, i] = 1
        elif (A[:,i] <= b):
            y_predicted[:, i] = 0
    
    return y_predicted

In [71]:
w0 = np.zeros((X_train_tr.shape[0], 1))

n_iterations = 2000
eta = 1.255

w, losses = optimize(w0, X_train_tr, y_train_tr, n_iterations, eta)
print(f"Итоговый вектор весов w: {w}")

Итоговый вектор весов w: [[ 0.3973941 ]
 [-0.3973941 ]
 [ 0.60583312]
 [-0.60583312]
 [ 0.21018302]
 [-0.15360763]
 [-0.11057133]
 [-0.14336311]
 [ 0.03134672]
 [-0.11788285]
 [ 0.02435563]
 [ 0.46081882]
 [-0.18262286]
 [-0.28326366]
 [ 0.10108537]
 [-0.08048057]
 [ 0.85709886]
 [ 0.11791341]
 [ 0.11448864]
 [ 0.37849405]
 [ 0.37684816]
 [ 0.1825071 ]
 [ 0.40782305]
 [ 0.15743216]
 [ 0.28800485]
 [ 0.15707682]
 [-0.34568453]]


In [72]:
y_predicted_train = predict(w, X_train_tr)

train_accuracy = 100.0 - np.mean(np.mean(np.abs(y_predicted_train - y_train_tr)*100.0))

print(f"Точность на обучающей выборке: {train_accuracy}")

Точность на обучающей выборке: 86.18914139287561


Проделаем всё тоже самое для Тестовой выборки

In [73]:
df_test = pd.read_csv(r"/content/passenger-satisfaction/pass_satisfaction_test.csv", sep=',')

X = df_test

transformer = make_column_transformer((OneHotEncoder(), X.select_dtypes(include='object').columns), remainder='passthrough')
transformed = transformer.fit_transform(X)
X = pd.DataFrame(transformed)

std = StandardScaler()
X_test = std.fit_transform(X)
X_test = pd.DataFrame(X_test, columns=X.columns)

X_test_tr = X_test.transpose()

y_predicted_test = predict(w, X_test_tr)
y_predicted_test = y_predicted_test.reshape(y_predicted_test.shape[1])
y_predicted_test = [int(i) for i in y_predicted_test]
y_predicted_test = label_encoder.inverse_transform(y_predicted_test)
coeff_df = pd.DataFrame({"id": df_test['id'].to_numpy(), "satisfaction": y_predicted_test})
coeff_df

,id,satisfaction
0,40853,neutral or dissatisfied
1,15070,satisfied
2,104309,neutral or dissatisfied
3,36427,neutral or dissatisfied
4,7630,satisfied
...,...,...
20776,16548,satisfied
20777,11384,satisfied
20778,58327,neutral or dissatisfied
20779,113956,satisfied


In [74]:
coeff_df.to_csv(r"/content/Predictions.csv", index=False)